# Traits

* Scala does not support multiple inheritance.
* Scala has traits similar to Java's interfaces. Traits can have
abstract and concrete methods as well as fields(state).
* Java interfaces can be used as scala traits.
* A class can implement multiple traits

In [1]:
// no abstract keyword required
// no override is required when implementing abstract methods of trait
trait Hashable {
    // methods with no definition are trait's abstract methods
    def hash: Long
}

defined trait Hashable

In [3]:
// we can implement multiple traits
class Base
class Derived extends Base 
                with Hashable 
                with Cloneable 
                with Serializable {
                    def hash = ???
}

defined class Base
defined class Derived

## Traits with concrete implementations

* Traits with concrete methods can act as mixin. On their own they
dont add value. When added to classes, they extend the class's functionality

In [4]:
trait Logger {
    def log(msg: String): Unit = { 
        println(msg) 
    }
}

defined trait Logger

In [10]:
//Adding trait to individual instances
class Item(val name: String)

def printMethods(obj: Item) {
    obj.getClass.getMethods.map(_.getName).distinct.foreach(println(_))
}

val item1 = new Item("a")
val item2 = new Item("b") with Logger
printMethods(item1)
println("=================")
printMethods(item2) // this object should contain log method

ammonite$$sess$cmd9$Helper$Item$$$outer
name
wait
equals
toString
hashCode
getClass
notify
notifyAll
ammonite$$sess$cmd3$Helper$Logger$$$outer
log
ammonite$$sess$cmd9$Helper$Item$$$outer
name
wait
equals
toString
hashCode
getClass
notify
notifyAll


defined class Item
defined function printMethods
item1: Item = ammonite.$sess.cmd9$Helper$Item@20715e36
item2: Item with Logger = ammonite.$sess.cmd9$Helper$$anon$1@51f5368f

* Traits can be layered, (i.e) subtrait can call method of the
base trait using `super`. Base of the trait that super resolves
to depends on the order in which the traits are added.

* When traits are ordered using `with trait1 with trait2`, the methods
are called from last to first, `trait2.someMethod` will be called which inturn
will call `trait1.someMethod` because of its `super` call. Imagine the traits
are stacked, with last one being called first.


* Traits can have concrete(initialized) and abstract fields(uninitialized)
* Fields can be val or var
* Each such field is added to the class using the trait by the scala compiler. 
 In the byte code of the class, we can observe the trait's fields added as the class's
 own fields.

## Trait construction order

* Traits cannot have constructor parameters but can have constructor body

In [11]:
trait SimpleTrait {
    // traits constructor body
    println("hello world")
    def someMethod: Int
}

defined trait SimpleTrait

Constructors execution order

* Superclass constructor
* Trait constructors executed from left to right.(Traits are constructed left to right, 
but when traits methods are invoked, method call (involving `super`) passes from right towards left)
* Within each trait, the construction is from the trait's parent to itself
* After all traits are constructed, the subclass is constructed

**NOTE**: If multiple traits share a common parent, and that parent has already been
constructed, it is not constructed again.

In [13]:
trait BaseTrait {
    println("Inside BaseTrait")
}

trait DerivedTrait1 extends BaseTrait {
    println("Inside Derived trait1")
}

trait DerivedTrait2 extends BaseTrait {
    println("Inside Derived trait2")
}

trait BaseTrait2{
    println("Inside BaseTrait2")
}

class Base {
    println("Inside Base class")
}

class Derived extends Base
                with DerivedTrait1
                with DerivedTrait2
                with BaseTrait2 {
                println("Inside Derived class")
}

// This will print the statement that help us understand
// the construction order
val derived = new Derived

Inside Base class
Inside BaseTrait
Inside Derived trait1
Inside Derived trait2
Inside BaseTrait2
Inside Derived class


defined trait BaseTrait
defined trait DerivedTrait1
defined trait DerivedTrait2
defined trait BaseTrait2
defined class Base
defined class Derived
derived: Derived = ammonite.$sess.cmd12$Helper$Derived@4b6537d0

**NOTE**: Linearization gives the order in which the super is resolved.
It is the reverse of the construction order.

```text
class C extends C1 with C2 ...with Cn
```
Then lin(C) = C -> lin(Cn) -> ... -> lin(C1). In case of duplicate members
rightmost one is kept.

```text
class SavingsAccount extends Account with FileLogger with ShortLogger

lin(SavingsAccount)
= SavingsAccount » lin(ShortLogger) » lin(FileLogger) » lin(Account)
# Here Logger is duplicated. So keep the rightmost one.
= SavingsAccount » (ShortLogger » Logger) » (FileLogger » Logger) » lin(Account)
= SavingsAccount » ShortLogger » FileLogger » Logger » Account.
```

## Trait extending classes

* Traits can extend classes
* A class that extends such a trait will have trait's superclass
as its superclass.
* Such subclasses can extend from any other type that is either the same
as the superclass of the trait or a derived type of the trait's
superclass.

## Self Types

Using self types we can ensure that the particular trait can
be mixed into subclasses of a particular type.

In [14]:
trait SomeBaseTrait

trait ExceptionSelfTypeTrait extends SomeBaseTrait {
    // we can also have structural type for self type
    this: Exception =>
         //trait body
    
    //Within this trait, we can call any method,field of 
    // of the self type
    def someMethod: String = getMessage()
}

// ExceptionSelfTypeTrait can only be mixed into subclass of
// exception
class SomeException extends Exception
with ExceptionSelfTypeTrait

defined trait SomeBaseTrait
defined trait ExceptionSelfTypeTrait
defined class SomeException

Structural types - Types without a class name but only the members 
the class should have. Such structural types are used in the early
definition syntax

```Scala
trait SelfTypeTrait {
    // We can mix this trait into any class with a format method
    this: {def format(fmt: String): String } =>
        //traits body
}
```

In [15]:
class SomeBaseException extends Exception

// Extending from subclass of self type is also allowed.
class SomeException extends SomeBaseException 
with ExceptionSelfTypeTrait

defined class SomeBaseException
defined class SomeException

In [14]:
class NotAnException

// This will lead to compilation error
class SomeException extends NotAnException with ExceptionSelfTypeTrait

cmd14.sc:3: illegal inheritance;
 self-type Helper.this.SomeException does not conform to cmd14.this.cmd13.ExceptionSelfTypeTrait's selftype cmd14.this.cmd13.ExceptionSelfTypeTrait with Exception
class SomeException extends NotAnException with ExceptionSelfTypeTrait
                                                ^Compilation Failed

: 

To know how traits are being handled under the hood by scala compiler to JVM bytecode,
refer the book.